In [12]:
import tensorflow as tf
import argparse
import os.path
from cifar10_models import *
from cifar10_input import *
from utils import *
import numpy as np

tf.logging.set_verbosity(tf.logging.INFO)

#### SET THIS BEFORE RUNNING
NAME = "cifar10/cifar10_scm_001"
try:
    os.makedirs(CHECKPOINT.format(NAME))
except FileExistsError:
    pass

tf.reset_default_graph()
train_files, test_files, validation_files = maybe_download_and_extract()
train_data = Cifar10Record(train_files, epochs=1)
test_data = Cifar10Record(test_files, epochs=1)
validation_data = Cifar10Record(validation_files, epochs=1)

#model = Cifar10ShallowConvolutionalModel(test_data.image, test_data.label)
#model = Cifar10ShallowConvolutionalModel(train_data.image, train_data.label)
model = Cifar10ShallowConvolutionalModel(validation_data.image, validation_data.label)


try:
    checkpoints = sorted(["{}/{}".format(ROOT.format(NAME),x[:-5]) for x in os.listdir(ROOT.format(NAME)) if x.endswith(".meta")], 
                     key=lambda x: int(x.split("-")[-1]))
except FileNotFoundError:
    checkpoints = []
model.saver.recover_last_checkpoints(checkpoints)
try: latest_checkpoint = model.saver.last_checkpoints[-2]
except: latest_checkpoint = None

In [13]:
acc = []
with tf.Session() as sess:
    model.saver.restore(sess, latest_checkpoint)
    while True:
        try:
            acc.append(model.accuracy.eval())
        except Exception as e:
            print(e)
            break

INFO:tensorflow:Restoring parameters from ../models/cifar10/cifar10_scm_001/model.ckpt-864000
End of sequence
	 [[Node: IteratorGetNext_2 = IteratorGetNext[output_shapes=[[?,32,32,3], [?,10]], output_types=[DT_FLOAT, DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](OneShotIterator_2)]]

Caused by op 'IteratorGetNext_2', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.py", line 177, in start
    sup

In [14]:
print(len(acc),
    np.mean(acc))

100 0.6453001


In [15]:
print(len(acc), np.mean(acc))

100 0.6453001


In [16]:
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file

# List ALL tensors example output: v0/Adam (DT_FLOAT) [3,3,1,80]
print_tensors_in_checkpoint_file(file_name=latest_checkpoint, tensor_name='', all_tensors=True)

tensor_name:  cifar10_scm/layer_1_conv/bias
[ 0.15725261  0.09974834  0.22683863  0.20731027  0.06970217 -0.0126459
  0.16080582  0.18068016  0.3047176   0.3912322   0.09802096  0.13175277
  0.12357131  0.45683068  0.14965133  0.07747586  0.45734787  0.1827745
  0.14417791  0.01328151  0.15231149  0.12254275  0.31908533  0.17824097
  0.3167306   0.21108185  0.33558932  0.03515026  0.26932105  0.33820382
  0.39210698  0.32147515]
tensor_name:  cifar10_scm/layer_1_conv/weights
[[[[ 2.16756724e-02  2.15476468e-01 -9.06610861e-02 ...  2.64526438e-02
    -1.00966111e-01  2.67585576e-01]
   [-1.70064807e-01 -6.09575696e-02 -9.31331962e-02 ...  1.16409101e-01
     5.53453304e-02  6.36508316e-02]
   [-2.33990654e-01  6.53924197e-02 -1.78778525e-02 ...  3.60614173e-02
     2.40418911e-02  1.49463087e-01]]

  [[-2.68387347e-02  3.02471936e-01 -9.44690481e-02 ... -1.91734284e-01
     4.11064662e-02 -1.04681432e-01]
   [ 3.42739187e-02  1.89413011e-01  8.53428326e-04 ... -3.53242196e-02
    -1.796

In [17]:
import numpy as np
completed = np.load("completed_cifar10.npy")

FileNotFoundError: [Errno 2] No such file or directory: 'completed_cifar10.npy'